In [ ]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==23.12.* cuml-cu12==23.12.* cugraph-cu12==23.12.* \
    cuspatial-cu12==23.12.* cuproj-cu12==23.12.*

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.2/955.2 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 615.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cupy, cuml, cugraph, cuspatial, cudf

In [ ]:
cudf.__version__

In [ ]:
import base64
import os
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import numpy as np
from cuml import RandomForestClassifier as RandomForestClassifierGPU
import cupy as cp
from sklearn.ensemble import RandomForestClassifier
import time
import pandas as pd
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
import nltk
from sklearn.feature_extraction.text import CountVectorizer

def getStem(review):
    review = review.lower()
    tokens = tokenizer.tokenize(review)
    removed_stopwords = [w for w in tokens if w not in sw]
    stemmed_words = [ps.stem(token) for token in removed_stopwords]
    clean_review = ' '.join(stemmed_words)
    return clean_review

def getDoc(document):
  d = []
  for doc in document:
      d.append(getStem(doc))
  return d

nltk.download('stopwords')
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

data=pd.read_csv('spam.csv')
data['Spam']=data['Category'].apply(lambda x:1 if x=='spam' else 0)
data2 = data.to_numpy()
X2 = data2[:, 1]
stemmed_doc = getDoc(X2)
cv = CountVectorizer()
vc = cv.fit_transform(stemmed_doc)
X2 = vc.todense()

CLIENT_SECRET_FILE = '' # insert your json here
API_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly'] #full acces to gmail mailbox

MODEL_FILE = 'modelspamcuda.pkl'

def prepare(messages):
    d = getDoc(messages)
    print(d)
    return cv.transform(d)

def authenticate_gmail():
    creds = None
    token_file = 'token.json'

    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(token_file, 'w') as token:
            token.write(creds.to_json())

    return creds


def read_email():
    creds = authenticate_gmail()
    service = build(API_NAME, API_VERSION, credentials=creds)

    # Pobierz listę wiadomości
    results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
    messages = results.get('messages', [])


    if not messages:
        print('Brak nowych wiadomości.')
    else:
        msg = service.users().messages().get(userId='me', id=messages[0]['id']).execute()
        is_spam = classify_spam(msg)

        if is_spam:
          print(f'Message is classified as spam. Move to SPAM folder.')
        else:
          print(f'Message is not classified as spam. Move to THRASH folder.')

def move_to_spam(email_id, decision):
    creds = authenticate_gmail()
    service = build(API_NAME, API_VERSION, credentials=creds)

    if (decision == 1): spam_folder_id = "SPAM"
    else: spam_folder_id = "TRASH"

    print(spam_folder_id)
    print(email_id)
    service.users().messages().modify(userId='me', id=email_id, body={'addLabelIds': [spam_folder_id]}).execute()

def classify_spam(msg):

  with open(MODEL_FILE, 'rb') as model_file:
    model = pickle.load(model_file)
    messages = prepare([msg['snippet']])
    X3 = messages.todense()
    X_gpu = cp.array(X3)
    y_pred = model.predict(X_gpu)
    move_to_spam(msg['id'], int(y_pred))
    return bool(y_pred)


def list_labels():
    creds = authenticate_gmail()
    service = build(API_NAME, API_VERSION, credentials=creds)

    labels = service.users().labels().list(userId='me').execute()

    if 'labels' in labels:
        for label in labels['labels']:
            print(f"Identyfikator: {label['id']}, Nazwa: {label['name']}")

if __name__ == '__main__':
    read_email()
    # list_labels()
